<a href="https://colab.research.google.com/github/Asif1310/3004/blob/main/Convolutional_neural_network_for_face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the required libraries

In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import zipfile
import os

# Importing the contents of github repository

In [2]:
!wget -O '/content/main.zip' 'https://github.com/Asif1310/3004/archive/refs/heads/main.zip'
zip = zipfile.ZipFile('/content/main.zip','r')
zip.extractall('/content')
zip.close()

--2022-02-20 14:08:59--  https://github.com/Asif1310/3004/archive/refs/heads/main.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Asif1310/3004/zip/refs/heads/main [following]
--2022-02-20 14:08:59--  https://codeload.github.com/Asif1310/3004/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/content/main.zip’

/content/main.zip       [      <=>           ]  30.57M  26.0MB/s    in 1.2s    

2022-02-20 14:09:01 (26.0 MB/s) - ‘/content/main.zip’ saved [32053978]



## Extracting the data folder 

In [3]:
zip = zipfile.ZipFile('/content/3004-main/data.zip','r')
zip.extractall('/content')
zip.close()

## Extracting the animated folder

In [4]:
zip = zipfile.ZipFile('/content/3004-main/animated.zip','r')
zip.extractall('/content')
zip.close()

# Preprocessing the images:

i) Training set

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/training',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 277 images belonging to 2 classes.


ii) Test set

In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        '/content/test',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 30 images belonging to 2 classes.


# Building the model

In [7]:
model = tf.keras.models.Sequential() #creating an instance of Sequential class 
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3))) #first convolutional layer
model.add(tf.keras.layers.MaxPooling2D((2, 2))) #first sub-sampling layer
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu')) #second convolutional layer
model.add(tf.keras.layers.MaxPooling2D((2, 2))) #second sub-sampling layer
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu')) #third convolutional layer

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        36928     
                                                                 
Total params: 56,320
Trainable params: 56,320
Non-traina

Creating a flatten layer to convert the input into a vector

In [9]:
model.add(tf.keras.layers.Flatten())

Creating the fully connected layer and the final output layer 

In [10]:
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#Compiling and training the model:

In [11]:
model.compile(optimizer='adam',
              loss= 'binary_crossentropy',
              metrics=['accuracy'])

In [24]:
model.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
9/9 [==============================] - 5s 518ms/step - loss: 0.0667 - accuracy: 0.9783 - val_loss: 0.3743 - val_accuracy: 0.8667
Epoch 2/25
9/9 [==============================] - 5s 487ms/step - loss: 0.0506 - accuracy: 0.9892 - val_loss: 0.5681 - val_accuracy: 0.8000
Epoch 3/25
9/9 [==============================] - 5s 487ms/step - loss: 0.1187 - accuracy: 0.9747 - val_loss: 0.3996 - val_accuracy: 0.8667
Epoch 4/25
9/9 [==============================] - 5s 518ms/step - loss: 0.0837 - accuracy: 0.9783 - val_loss: 0.4197 - val_accuracy: 0.8333
Epoch 5/25
9/9 [==============================] - 5s 489ms/step - loss: 0.0633 - accuracy: 0.9711 - val_loss: 0.3593 - val_accuracy: 0.8667
Epoch 6/25
9/9 [==============================] - 4s 494ms/step - loss: 0.0585 - accuracy: 0.9747 - val_loss: 0.2780 - val_accuracy: 0.9000
Epoch 7/25
9/9 [==============================] - 5s 508ms/step - loss: 0.0863 - accuracy: 0.9711 - val_loss: 0.1854 - val_accuracy: 0.9000
Epoch 8/25
9/9 [====

# Using the model to predict some images

Defining a function named 'load_predict' which upon passing the directory name as the parameter returns predictions on all the images stored by that directory

In [19]:
def load_predict(directory):
  prediction_names = []
  for i in os.listdir(directory):
    j = directory + i
    test1 = image.load_img(j, target_size = (64, 64))
    test1 = image.img_to_array(test1)
    test1 = np.expand_dims(test1, axis = 0)
    pred = model.predict(test1)
    if pred == 0:
      pred1 = "Ronaldo"
    else:
      pred1 = "messi"
    prediction_names.append(pred1)
  return prediction_names

i) Predictions on the test set of Ronaldo images

In [25]:
directory = '/content/test/class 0/'
pred = load_predict(directory)
print("Number of Ronaldo predictions : %2d" %(pred.count('Ronaldo')))
print("Number of Messi predictions : %2d" %(pred.count('messi')))

Number of Ronaldo predictions : 13
Number of Messi predictions :  2


ii) Predictions on the test set of Messi images

In [26]:
directory = '/content/test/class 1/'
pred = load_predict(directory)
print("Number of Ronaldo predictions : %2d" %(pred.count('Ronaldo')))
print("Number of Messi predictions : %2d" %(pred.count('messi')))

Number of Ronaldo predictions :  3
Number of Messi predictions : 12


iii) Predictions on the animated set of Ronaldo images

In [27]:
directory = '/content/Ronaldo_animated/'
pred = load_predict(directory)
print("Number of Ronaldo predictions : %2d" %(pred.count('Ronaldo')))
print("Number of Messi predictions : %2d" %(pred.count('messi')))

Number of Ronaldo predictions : 18
Number of Messi predictions :  7


iv) Predictions on the animated set of Messi images

In [28]:
directory = '/content/Messi_animated/'
pred = load_predict(directory)
print("Number of Ronaldo predictions : %2d" %(pred.count('Ronaldo')))
print("Number of Messi predictions : %2d" %(pred.count('messi')))

Number of Ronaldo predictions : 11
Number of Messi predictions : 14
